In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib
from joblib import dump, load

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
nltk.download('stopwords')
corpus = []
for i in range(len(train)):
    tweet = re.sub('[^a-zA-Z]', ' ', train['text'][i])
    tweet = tweet.lower()
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()

In [ ]:
lda = LDA(n_components=2, random_state=42)
lda.fit(X)

LatentDirichletAllocation(n_components=2, random_state=42)

In [ ]:
import pickle

# Saving the models
with open('cv.pkl', 'wb') as f:
    pickle.dump(cv, f)

with open('lda.pkl', 'wb') as f:
    pickle.dump(lda, f)

In [ ]:
X_lda = lda.transform(X)
train['topic'] = np.argmax(X_lda, axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, train['target'], test_size=0.2, random_state=42)


In [ ]:
xgb = XGBClassifier(n_estimators=200,learning_rate = 0.2,max_depth = 8)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
with open('xgb.pkl', 'wb') as f:
    pickle.dump(xgb, f)
dump(xgb, 'xgb.joblib')

['xgb.joblib']

In [ ]:
print(X_test)
y_pred = xgb.predict(X_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       874
           1       0.81      0.70      0.75       649

    accuracy                           0.80      1523
   macro avg       0.80      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523



In [ ]:
corpus_test = []
for i in range(len(test)):
    tweet = re.sub('[^a-zA-Z]', ' ', test['text'][i])
    tweet = tweet.lower()
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus_test.append(tweet)

In [ ]:
print(X_test)
X_test = cv.transform(corpus_test).toarray()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
X_test_lda = lda.transform(X_test)
test['topic'] = np.argmax(X_test_lda, axis=1)
y_pred_test = xgb.predict(X_test)

In [ ]:
test['target'] = y_pred_test
test[['id', 'target']].to_csv('submission.csv', index=False)